In [6]:
from pathlib import Path
import warnings
import os
warnings.filterwarnings("ignore")

import yaml
import datetime


import xarray as xr
import numpy as np
import pandas as pd


from torch.utils.data import DataLoader
import lightning.pytorch as pl

In [8]:
def calc_phase_number(phase):
    """Return DataArray of MJO phase number determined from phase angle. Note that
    phase number is calculated for all dates, regardless of MJO amplitude."""
    phase_number = phase.copy(deep=True).rename("phase_number")
    phase_number[:] = np.nan
    phase_number[(phase >= -180) & (phase < -135)] = 1
    phase_number[(phase >= -135) & (phase < -90)] = 2
    phase_number[(phase >= -90) & (phase < -45)] = 3
    phase_number[(phase >= -45) & (phase < 0)] = 4
    phase_number[(phase >= 0) & (phase < 45)] = 5
    phase_number[(phase >= 45) & (phase < 90)] = 6
    phase_number[(phase >= 90) & (phase < 135)] = 7
    phase_number[(phase >= 135) & (phase <= 180)] = 8
    return phase_number

## Extract RMM from .txt NOAA - file 

In [9]:
# set variable downloaded from the 20th century Reanalysis Data
vrbl = 'RMM'

cfd = Path(os.getcwd()).parent.absolute()
data_dir = str(Path(cfd).parent.absolute()) + '/Data/'
dataset_dir = f'{data_dir}Index/'
    
resolution = '1.40625' # in degrees, available: '5.625'

start_year, end_year = 1980, 2023 # set both to None to use all available years


reduce_to_single_dim = False # if True, the dataset will be reduced to a single dimension (time) by aggregating over the other dimensions
months_to_keep = [11, 12, 1, 2, 3]
start_season, end_season = '11-01', '04-05'

file_name = 'rMII_index_latest.txt'
file_out = f"rmm_index_{start_year}-{start_season}_{end_year}-{end_season}.nc"

In [10]:
# Load as numpy array.
X = np.loadtxt(dataset_dir + file_name)

### Format in dataset (xarray)

In [11]:
datetimes = []
rmm1 = []
rmm2 = []
indexs = []

for i in range(len(X)):
    datetimes.append(np.datetime64(datetime.date(int(X[i][0]), int(X[i][1]), int(X[i][2]))))
    rmm1.append(X[i][3])
    rmm2.append(X[i][4])
    indexs.append(X[i][5])


rmm1 = np.array(rmm1)
rmm2 = np.array(rmm2)
indexs = np.array(indexs)

In [12]:
ds = xr.Dataset(
    data_vars=dict(
        rmm1=(["time"], rmm1),
        rmm2=(["time"], rmm2),
        noaa_index = (["time"], indexs),
    ),
    coords=dict(
        time=datetimes,
    ),
    attrs=dict(description="RMM index from NOAA data updated 04.09.2023"),
)

ds_time = ds.sel({'time':slice(np.datetime64(f'{start_year}-{start_season}'),np.datetime64(f'{end_year}-{end_season}'))})
ds_time.to_netcdf(dataset_dir + file_out)
print(ds_time)

<xarray.Dataset> Size: 496kB
Dimensions:     (time: 15496)
Coordinates:
  * time        (time) datetime64[s] 124kB 1980-11-01 1980-11-02 ... 2023-04-05
Data variables:
    rmm1        (time) float64 124kB -0.4265 -0.5911 -0.7061 ... 0.0 0.0 0.0
    rmm2        (time) float64 124kB 1.336 1.239 1.11 0.9645 ... 0.0 0.0 0.0 0.0
    noaa_index  (time) float64 124kB 7.0 7.0 7.0 7.0 8.0 ... 5.0 5.0 5.0 5.0 5.0
Attributes:
    description:  RMM index from NOAA data updated 04.09.2023


## Calculation of RMM1/2 

In [13]:
infile = f"{dataset_dir}{file_out}"#Path to netcdf file produced by Extravert containing daily MJO RMM indices.
ndays = 7 #Width of running mean (in days).
timedim = "time"
outfile = f"{dataset_dir}mjo_rmm_index_noaa_{start_year}-{start_season}_{end_year}-{end_season}.nc"#Path for output netcdf file.

In [14]:
# Read data
ds = xr.open_dataset(infile, engine ="netcdf4")

time_coords = ds.time.values[int(ndays/2):-int(ndays/2)]
# Calculate rolling average
avg = ds.rolling(**{timedim:ndays, "center":True}).mean(skipna=True)

avg = avg.sel(time=slice(avg.time.values[int(ndays/2)], avg.time.values[-int(ndays/2)-1])).assign_coords(time=time_coords)


# Recalculate properties time-averaged RMM1 and RMM2
phase = (np.arctan2(avg.rmm2, avg.rmm1) * 180 / np.pi).dropna("time")
amplitude = (np.sqrt(avg.rmm1 * avg.rmm1 + avg.rmm2 * avg.rmm2)).dropna("time")
phase_number = calc_phase_number(phase).dropna("time")

# Insert values back into avg dataset
avg = avg.assign(phase = phase)
avg = avg.assign(amplitude = amplitude)
avg = avg.assign(phase_number = phase_number)


avg.to_netcdf(outfile)

## Calculation of MJO Index

In [ ]:
# set variable downloaded from the 20th century Reanalysis Data
vrbl = 'MJO'

data_name = 'ERA5'

# var_index = 'phase_number'
var_index = 'rmm1' 
# var_index = 'rmm2'

dataset_dir = Path(f'{data_dir}{data_name}/datasets')
    
resolution = '1.40625' # in degrees, available: '5.625'
if 'ERA5' in data_name:
    start_year, end_year = 1980, 2023 # set both to None to use all available years
else:
    start_year, end_year = 1836, 1980 # set both to None to use all available years


reduce_to_single_dim = False # if True, the dataset will be reduced to a single dimension (time) by aggregating over the other dimensions
months_to_keep = [11, 12, 1, 2, 3]
start_season, end_season = '11-15', '03-31'


var_name = 'SST' # available: 't2m', 'tp', 'z'
region = 'mjo' # available: 'global', 'europe', 'northern_hemi'
pressure_level = '' # in hPa, ignored for variables without levels. available: 50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1,000
days =7


region_coords = {
    'spv': {'lat': slice(61,59)}, # u10 for Polar Vortex
    'mjo': None, # index for MJO
    }

index_path = f'{data_dir}/Index'


In [ ]:

# index_name = index_path / Path(outfile)
ds = xr.open_dataset(outfile)
ds = ds.sel({'time':slice(np.datetime64(f'{start_year}-{start_season}'),np.datetime64(f'{end_year}-{end_season}'))})
ds = ds.sel(time=ds['time.month'].isin(months_to_keep))


In [33]:
# compute seasons coordinates
start_season, end_season = '11-15', '03-31'

yrs = np.arange(start_year, end_year+1)
for i in range(len(yrs)-1):
    year = yrs[i]
    ds_sel = ds.sel({'time':slice(np.datetime64(f'{year}-{start_season}'),np.datetime64(f'{year+1}-{end_season}'))}).resample(time='1D').mean('time')
    len_time = len(ds_sel.time.values)
    seas_coord = np.repeat(i+1,len_time)
    ds_sel = ds_sel.assign_coords(season=('time', seas_coord))
    
    if i > 0:
        dst = xr.concat((dst,ds_sel), dim ='time')
        seas.append(len_time)
    if i ==0:
        seas = [len_time,len_time]
        dst = ds_sel
        del ds_sel
print(f'Season assignment done.')    

ds = dst[var_index]
del ds_sel, dst



if var_index == 'phase_number':
    ds.to_netcdf(f"{dataset_dir}/{vrbl}_index_{start_year}-{end_year}_{data_name}.nc")
else:
    ds.to_netcdf(f"{dataset_dir}/{var_index}_index_{start_year}-{end_year}_{data_name}.nc")

Season assignment done.


## Check regime length alignment.

In [34]:
nae = xr.open_dataarray(f"{dataset_dir}/z_500_1.40625deg_1980-2023_northern_hemi_2d_NAEregimes.nc")
categories_nae = nae.values.astype(int)
ds_norm = nae.expand_dims(dim={'nae_regime_cat': int(np.max(categories_nae)+1)}).transpose('time', 'nae_regime_cat')
ds_norm.copy(data=np.eye(max(categories_nae)+1)[categories_nae])

<xarray.DataArray 'nae' (time: 5901, nae_regime_cat: 4)> Size: 189kB
array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], shape=(5901, 4))
Coordinates:
  * time     (time) datetime64[ns] 47kB 1980-11-15 1980-11-16 ... 2023-03-31
    season   (time) int64 47kB ...
Dimensions without coordinates: nae_regime_cat
Attributes:
    description:  Time series of North atlantic weather regimes. For each dat...

### Check even shape

In [35]:
categories = ds.values
print(f"{vrbl}: {categories.shape}")
print(f"nae: {categories_nae.shape}")


MJO: (5901,)
nae: (5901,)


In [36]:
if var_index == 'phase_number':
    xr.load_dataarray(f"{dataset_dir}/{vrbl}_index_{start_year}-{end_year}_{data_name}.nc")
else:
    xr.load_dataarray(f"{dataset_dir}/{var_index}_index_{start_year}-{end_year}_{data_name}.nc")


## Format according to Dataset

In [37]:
class ClimateIndicies(pl.LightningDataModule):
    def __init__(self, ds, data_info, seasons=None, return_dates=False):
        self.lag = data_info['config'].get('n_steps_lag')
        self.n_in = data_info['config'].get('n_steps_in')
        self.n_out = data_info['config'].get('n_steps_out')
        self.stack_maps = data_info['config'].get('stack_maps')
        self.regime_path = data_info['config'].get('regime_path','')
        self.data_path = data_info['config'].get('data_path','')
        self.strt = data_info['config'].get('strt','1950')
        self.return_dates = return_dates
        self.seasons = seasons

        self.inputs = []
        self.time_steps = set()
        self.output = None

        if self.seasons is not None:
            ds = ds.sel(time=ds.season.isin(self.seasons))
        else:
            self.seasons=np.unique(ds.season.values)
        try:
            ds = ds[var_name].squeeze()
        except:
            ds = ds.squeeze()

        self.time_steps.add(len(ds.time))
            
        self.inputs.append(ds) 
        self.output = ds 

        
        # compute input and output shapes for model
        output_shape = (self.n_out, 1)
        self.shapes = {'input': [], 'output': output_shape}
        for data in self.inputs:
            self.shapes['input'].append((self.n_in, 1,))

        # compute number of samples
        assert len(self.time_steps) == 1, 'All variables must have the same number of time steps'
        self.n_samples_per_season = {}
        for season in self.seasons:
            s = self.output.time.sel(time=self.output.season == season)
            self.n_samples_per_season[season] = max(len(s) - (self.n_in + self.lag + self.n_out) * 7, 0)

        self.n_samples_per_season_accumulated = [sum(list(self.n_samples_per_season.values())[:i]) for i in range(1,len(self.seasons)+1)]
            
    
    def __getitem__(self, idx):
        inputs = []
        season_idx = np.digitize(idx, self.n_samples_per_season_accumulated)    
        season = self.seasons[season_idx]
        idx = idx - self.n_samples_per_season_accumulated[season_idx-1] if season_idx > 0 else idx
        in_idxs = [idx + i*7 for i in range(self.n_in)]
        out_idxs = [idx + (self.n_in + self.lag + i) * 7 for i in range(self.n_out)]

        for d in self.inputs:
            inp = d.sel(time=d.season == season).isel(time=in_idxs).values
            inputs.append(inp)

        output_slice = self.output.sel(time=self.output.season == season).isel(time=out_idxs)
        output = output_slice.values

        if self.return_dates:
            dates = [datetime.datetime.fromisoformat((str(d)[:10])).timetuple().tm_yday for d in output_slice.time.values]
            r = (inputs, output, dates, output_slice.time.values)
        else:
            r = (inputs, output)

        return r

In [38]:
class DataLoader(pl.LightningDataModule):

    def __init__(self, dataset, data = None, batchsize = 32, **params):
        
        super().__init__()
        
        self.ds = dataset
        self.bs = batchsize
        self.dataset = {'train': [], 'val': [],'test': []}
        self.seasons = params.get('seasons',None) 
        self.return_dates = params.get('return_dates', False)
        self.combine_test = params.get('combine_test',False)
        if data is None: 
            raise Exception("Weather variable need to be specified")
        else:
            self.data = data

    def train_dataloader(self):
        self.dataset['train']= ClimateIndicies(
                    ds=self.ds,
                    data_info=self.data,
                    seasons=self.seasons['train']
                )
        return DataLoader(self.dataset['train'], batch_size = self.bs, shuffle=True)
    
    def val_dataloader(self):
        self.dataset['val']= ClimateIndicies(
                    ds=self.ds,
                    data_info=self.data,
                    seasons=self.seasons['val']
                )
        return DataLoader(self.dataset['val'], batch_size = self.bs, shuffle=False)
    
    def test_dataloader(self):
        self.dataset['test'] = ClimateIndicies(
                    ds=self.ds,
                    data_info=self.data,
                    seasons=self.seasons['test']
                )
            
        return DataLoader(self.dataset['test'], batch_size = self.bs, shuffle=False)
    
    def access_dataset(self):
        return self.dataset


In [39]:
from deepS2S.utils.utils import statics_from_config

cfile = '_index_lstm'
config = yaml.load(open(f'{cfd}/config/config{cfile}.yaml'), Loader=yaml.FullLoader)
config['data_root'] = str(cfd.parent.absolute()) + f'/Data'

data_info, seasons = statics_from_config(config)

seasons =  {'train':list(range(config['data']['fine']['train_start'], config['data']['fine']['train_end'])),
               'val':list(range(config['data']['fine']['val_start'], config['data']['fine']['val_end'])),
               'test':list(range(config['data']['fine']['test_start'], config['data']['fine']['test_end'])),}


params = {'seasons': seasons, 'test_set_name':data_name}


In [40]:
dates = []
daytimes = []
target_index = []
input_index = []
error_sum = 0
i = 0
num_smps = 0

if vrbl == 'MJO':
        test_set = ClimateIndicies(
                ds=ds,
                data_info=data_info,
                seasons=seasons['test'],
                return_dates=True)
else:
        test_set = ClimateIndicies(
        ds=ds,
        data_info=data_info,
        seasons=seasons['test'],
        return_dates=True)

for input, output, weeks, days in test_set:
    target_index.append(np.array(output).squeeze())
    input_index.append(np.array(input).squeeze())
    dates.append(np.array(weeks).squeeze())
    daytimes.append(np.array(days).squeeze())
    i += 1
    num_smps += len(np.array(output))

In [41]:
for var in ['dates', 'daytimes', 'target_index', 'input_index']:
    locals()[var] = np.concatenate(locals()[var]).reshape((int(num_smps/data_info['config'].get('n_steps_in')),data_info['config'].get('n_steps_in')))

if var_index == 'phase_number':
    np.savez(f"{index_path}/{vrbl}_index_{start_year}-{end_year}_{region}_testset.npz", input = input_index, 
         output = target_index, dates = dates, daytimes = daytimes)
else:
    np.savez(f"{index_path}/{var_index}_index_{start_year}-{end_year}_{region}_testset.npz", input = input_index, 
         output = target_index, dates = dates, daytimes = daytimes)
